In [1]:
import tensorflow as tf
import tensorflow_addons as tfa

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

In [2]:
def download_nmt():
#     path_to_zip = tf.keras.utils.get_file(
#     'spa-eng.zip', origin='http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip',
#     extract=True)

#     path_to_file = os.path.dirname(path_to_zip)+"/spa-eng/spa.txt"
    #path_to_file = "./ace-ind/datates.txt"
    path_to_file = "./ace-ind/datates.txt"
#     path_to_file = "./spa-eng/spa.txt"
    return path_to_file

In [3]:
class NMTDataset:
    def __init__(self, problem_type='id-ace'):
        self.problem_type = 'id-ace'
        self.inp_lang_tokenizer = None
        self.targ_lang_tokenizer = None
    

    def unicode_to_ascii(self, s):
        return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

    ## Step 1 and Step 2 
    def preprocess_sentence(self, w):
        w = self.unicode_to_ascii(w.lower().strip())

        # creating a space between a word and the punctuation following it
        # eg: "he is a boy." => "he is a boy ."
        # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
        w = re.sub(r"([?.!,¿])", r" \1 ", w)
        w = re.sub(r'[" "]+', " ", w)

        # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
        w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

        w = w.strip()

        # adding a start and an end token to the sentence
        # so that the model know when to start and stop predicting.
        w = '<start> ' + w + ' <end>'
        return w
    
    def create_dataset(self, path, num_examples):
        # path : path to spa-eng.txt file
        # num_examples : Limit the total number of training example for faster training (set num_examples = len(lines) to use full data)
        lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
        word_pairs = [[self.preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]

        return zip(*word_pairs)

    # Step 3 and Step 4
    def tokenize(self, lang):
        # lang = list of sentences in a language
        
        # print(len(lang), "example sentence: {}".format(lang[0]))
        lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<OOV>')
        lang_tokenizer.fit_on_texts(lang)

        ## tf.keras.preprocessing.text.Tokenizer.texts_to_sequences converts string (w1, w2, w3, ......, wn) 
        ## to a list of correspoding integer ids of words (id_w1, id_w2, id_w3, ...., id_wn)
        tensor = lang_tokenizer.texts_to_sequences(lang) 

        ## tf.keras.preprocessing.sequence.pad_sequences takes argument a list of integer id sequences 
        ## and pads the sequences to match the longest sequences in the given input
        tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

        return tensor, lang_tokenizer

    def load_dataset(self, path, num_examples=None):
        # creating cleaned input, output pairs
        targ_lang, inp_lang = self.create_dataset(path, num_examples)

        input_tensor, inp_lang_tokenizer = self.tokenize(inp_lang)
        target_tensor, targ_lang_tokenizer = self.tokenize(targ_lang)

        return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

    def call(self, num_examples, BUFFER_SIZE, BATCH_SIZE):
        file_path = download_nmt()
        input_tensor, target_tensor, self.inp_lang_tokenizer, self.targ_lang_tokenizer = self.load_dataset(file_path, num_examples)
        
        input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

        train_dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train))
        train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

        val_dataset = tf.data.Dataset.from_tensor_slices((input_tensor_val, target_tensor_val))
        val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=True)

        return train_dataset, val_dataset, self.inp_lang_tokenizer, self.targ_lang_tokenizer

In [4]:
BUFFER_SIZE = 650
BATCH_SIZE = 32
# Let's limit the #training examples for faster training
num_examples = 1300

dataset_creator = NMTDataset('id-ace')
train_dataset, val_dataset, inp_lang, targ_lang = dataset_creator.call(num_examples, BUFFER_SIZE, BATCH_SIZE)

In [5]:
example_input_batch, example_target_batch = next(iter(train_dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([32, 11]), TensorShape([32, 12]))

In [6]:
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1
max_length_input = example_input_batch.shape[1]
max_length_output = example_target_batch.shape[1]

embedding_dim = 256
units = 1300
steps_per_epoch = num_examples//BATCH_SIZE

In [7]:
print("max_length_aceh, max_length_indonesia, vocab_size_aceh, vocab_size_indonesia")
max_length_input, max_length_output, vocab_inp_size, vocab_tar_size

max_length_aceh, max_length_indonesia, vocab_size_aceh, vocab_size_indonesia


(11, 12, 697, 811)

In [8]:
##### 

class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)

    ##-------- LSTM layer in Encoder ------- ##
    self.lstm_layer = tf.keras.layers.LSTM(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    


  def call(self, x, hidden):
    x = self.embedding(x)
    output, h, c = self.lstm_layer(x, initial_state = hidden)
    return output, h, c

  def initialize_hidden_state(self):
    return [tf.zeros((self.batch_sz, self.enc_units)), tf.zeros((self.batch_sz, self.enc_units))] 

In [9]:
## Test Encoder Stack

encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)


# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_h, sample_c = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder h vecotr shape: (batch size, units) {}'.format(sample_h.shape))
print ('Encoder c vector shape: (batch size, units) {}'.format(sample_c.shape))

Encoder output shape: (batch size, sequence length, units) (32, 11, 1300)
Encoder h vecotr shape: (batch size, units) (32, 1300)
Encoder c vector shape: (batch size, units) (32, 1300)


In [10]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz, attention_type='luong'):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.attention_type = attention_type
    
    # Embedding Layer
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    
    #Final Dense layer on which softmax will be applied
    self.fc = tf.keras.layers.Dense(vocab_size)

    # Define the fundamental cell for decoder recurrent structure
    self.decoder_rnn_cell = tf.keras.layers.LSTMCell(self.dec_units)
   


    # Sampler
    self.sampler = tfa.seq2seq.sampler.TrainingSampler()

    # Create attention mechanism with memory = None
    self.attention_mechanism = self.build_attention_mechanism(self.dec_units, 
                                                              None, self.batch_sz*[max_length_input], self.attention_type)

    # Wrap attention mechanism with the fundamental rnn cell of decoder
    self.rnn_cell = self.build_rnn_cell(batch_sz)

    # Define the decoder with respect to fundamental rnn cell
    self.decoder = tfa.seq2seq.BasicDecoder(self.rnn_cell, sampler=self.sampler, output_layer=self.fc)

    
  def build_rnn_cell(self, batch_sz):
    rnn_cell = tfa.seq2seq.AttentionWrapper(self.decoder_rnn_cell, 
                                  self.attention_mechanism, attention_layer_size=self.dec_units)
    return rnn_cell

  def build_attention_mechanism(self, dec_units, memory, memory_sequence_length, attention_type='luong'):
    # ------------- #
    # typ: Which sort of attention (Bahdanau, Luong)
    # dec_units: final dimension of attention outputs 
    # memory: encoder hidden states of shape (batch_size, max_length_input, enc_units)
    # memory_sequence_length: 1d array of shape (batch_size) with every element set to max_length_input (for masking purpose)

    if(attention_type=='bahdanau'):
      return tfa.seq2seq.BahdanauAttention(units=dec_units, memory=memory, memory_sequence_length=memory_sequence_length)
    else:
      return tfa.seq2seq.LuongAttention(units=dec_units, memory=memory, memory_sequence_length=memory_sequence_length)

  def build_initial_state(self, batch_sz, encoder_state, Dtype):
    decoder_initial_state = self.rnn_cell.get_initial_state(batch_size=batch_sz, dtype=Dtype)
    decoder_initial_state = decoder_initial_state.clone(cell_state=encoder_state)
    return decoder_initial_state


  def call(self, inputs, initial_state):
    x = self.embedding(inputs)
    outputs, _, _ = self.decoder(x, initial_state=initial_state, sequence_length=self.batch_sz*[max_length_output-1])
    return outputs


In [11]:
# Test decoder stack

decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE, 'luong')
sample_x = tf.random.uniform((BATCH_SIZE, max_length_output))
decoder.attention_mechanism.setup_memory(sample_output)
initial_state = decoder.build_initial_state(BATCH_SIZE, [sample_h, sample_c], tf.float32)


sample_decoder_outputs = decoder(sample_x, initial_state)

print("Decoder Outputs Shape: ", sample_decoder_outputs.rnn_output.shape)


Decoder Outputs Shape:  (32, 11, 811)


In [12]:
optimizer = tf.keras.optimizers.Adam()


def loss_function(real, pred):
  # real shape = (BATCH_SIZE, max_length_output)
  # pred shape = (BATCH_SIZE, max_length_output, tar_vocab_size )
  cross_entropy = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
  loss = cross_entropy(y_true=real, y_pred=pred)
  mask = tf.logical_not(tf.math.equal(real,0))   #output 0 for y=0 else output 1
  mask = tf.cast(mask, dtype=loss.dtype)  
  loss = mask* loss
  loss = tf.reduce_mean(loss)
  return loss  

In [13]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [14]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_h, enc_c = encoder(inp, enc_hidden)


    dec_input = targ[ : , :-1 ] # Ignore <end> token
    real = targ[ : , 1: ]         # ignore <start> token

    # Set the AttentionMechanism object with encoder_outputs
    decoder.attention_mechanism.setup_memory(enc_output)

    # Create AttentionWrapperState as initial_state for decoder
    decoder_initial_state = decoder.build_initial_state(BATCH_SIZE, [enc_h, enc_c], tf.float32)
    pred = decoder(dec_input, decoder_initial_state)
    logits = pred.rnn_output
    loss = loss_function(real, logits)

  variables = encoder.trainable_variables + decoder.trainable_variables
  gradients = tape.gradient(loss, variables)
  optimizer.apply_gradients(zip(gradients, variables))

  return loss

In [15]:
EPOCHS = 500

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0
  # print(enc_hidden[0].shape, enc_hidden[1].shape)

  for (batch, (inp, targ)) in enumerate(train_dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.5f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.5f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 3.31212
Epoch 1 Loss 2.03915
Time taken for 1 epoch 33.215601205825806 sec

Epoch 2 Batch 0 Loss 2.09589
Epoch 2 Loss 1.41118
Time taken for 1 epoch 27.759023904800415 sec

Epoch 3 Batch 0 Loss 1.58084
Epoch 3 Loss 1.23032
Time taken for 1 epoch 26.963063716888428 sec

Epoch 4 Batch 0 Loss 1.39971
Epoch 4 Loss 1.14108
Time taken for 1 epoch 27.77260398864746 sec

Epoch 5 Batch 0 Loss 1.18298
Epoch 5 Loss 1.14715
Time taken for 1 epoch 27.521191596984863 sec

Epoch 6 Batch 0 Loss 1.24860
Epoch 6 Loss 0.96399
Time taken for 1 epoch 28.049736738204956 sec

Epoch 7 Batch 0 Loss 1.23886
Epoch 7 Loss 0.83249
Time taken for 1 epoch 28.13473868370056 sec

Epoch 8 Batch 0 Loss 0.94425
Epoch 8 Loss 0.72145
Time taken for 1 epoch 28.260355234146118 sec

Epoch 9 Batch 0 Loss 0.71283
Epoch 9 Loss 0.61268
Time taken for 1 epoch 26.946061372756958 sec

Epoch 10 Batch 0 Loss 0.61457
Epoch 10 Loss 0.52458
Time taken for 1 epoch 27.49618124961853 sec

Epoch 11 Batch 0 Loss 0.54170
E

Epoch 84 Loss 0.00411
Time taken for 1 epoch 27.482213973999023 sec

Epoch 85 Batch 0 Loss 0.00204
Epoch 85 Loss 0.00364
Time taken for 1 epoch 26.83824324607849 sec

Epoch 86 Batch 0 Loss 0.00031
Epoch 86 Loss 0.00411
Time taken for 1 epoch 27.41282844543457 sec

Epoch 87 Batch 0 Loss 0.00172
Epoch 87 Loss 0.00326
Time taken for 1 epoch 26.83372664451599 sec

Epoch 88 Batch 0 Loss 0.00089
Epoch 88 Loss 0.00380
Time taken for 1 epoch 27.46448278427124 sec

Epoch 89 Batch 0 Loss 0.00709
Epoch 89 Loss 0.00408
Time taken for 1 epoch 26.821568965911865 sec

Epoch 90 Batch 0 Loss 0.00317
Epoch 90 Loss 0.00324
Time taken for 1 epoch 27.491870641708374 sec

Epoch 91 Batch 0 Loss 0.00056
Epoch 91 Loss 0.00324
Time taken for 1 epoch 26.80821204185486 sec

Epoch 92 Batch 0 Loss 0.00443
Epoch 92 Loss 0.00347
Time taken for 1 epoch 27.44362735748291 sec

Epoch 93 Batch 0 Loss 0.00284
Epoch 93 Loss 0.00363
Time taken for 1 epoch 26.807637453079224 sec

Epoch 94 Batch 0 Loss 0.00025
Epoch 94 Loss 0.

Epoch 166 Loss 0.00370
Time taken for 1 epoch 27.413549184799194 sec

Epoch 167 Batch 0 Loss 0.00235
Epoch 167 Loss 0.00358
Time taken for 1 epoch 26.853795051574707 sec

Epoch 168 Batch 0 Loss 0.00337
Epoch 168 Loss 0.00376
Time taken for 1 epoch 27.442888498306274 sec

Epoch 169 Batch 0 Loss 0.00020
Epoch 169 Loss 0.00370
Time taken for 1 epoch 26.734866857528687 sec

Epoch 170 Batch 0 Loss 0.00021
Epoch 170 Loss 0.00337
Time taken for 1 epoch 27.437012434005737 sec

Epoch 171 Batch 0 Loss 0.01125
Epoch 171 Loss 0.00334
Time taken for 1 epoch 26.784659385681152 sec

Epoch 172 Batch 0 Loss 0.00188
Epoch 172 Loss 0.00314
Time taken for 1 epoch 27.42556405067444 sec

Epoch 173 Batch 0 Loss 0.00419
Epoch 173 Loss 0.00340
Time taken for 1 epoch 26.866279125213623 sec

Epoch 174 Batch 0 Loss 0.00123
Epoch 174 Loss 0.00347
Time taken for 1 epoch 27.424052953720093 sec

Epoch 175 Batch 0 Loss 0.00335
Epoch 175 Loss 0.00354
Time taken for 1 epoch 26.822721481323242 sec

Epoch 176 Batch 0 Loss

Epoch 248 Batch 0 Loss 0.00607
Epoch 248 Loss 0.00779
Time taken for 1 epoch 27.45802593231201 sec

Epoch 249 Batch 0 Loss 0.01069
Epoch 249 Loss 0.00660
Time taken for 1 epoch 26.8322856426239 sec

Epoch 250 Batch 0 Loss 0.00103
Epoch 250 Loss 0.00467
Time taken for 1 epoch 27.49780249595642 sec

Epoch 251 Batch 0 Loss 0.00079
Epoch 251 Loss 0.00540
Time taken for 1 epoch 26.843482732772827 sec

Epoch 252 Batch 0 Loss 0.00419
Epoch 252 Loss 0.00497
Time taken for 1 epoch 27.621870040893555 sec

Epoch 253 Batch 0 Loss 0.00316
Epoch 253 Loss 0.00494
Time taken for 1 epoch 26.859915256500244 sec

Epoch 254 Batch 0 Loss 0.00260
Epoch 254 Loss 0.00496
Time taken for 1 epoch 27.435626983642578 sec

Epoch 255 Batch 0 Loss 0.00628
Epoch 255 Loss 0.00445
Time taken for 1 epoch 26.886560201644897 sec

Epoch 256 Batch 0 Loss 0.00103
Epoch 256 Loss 0.00417
Time taken for 1 epoch 27.534595012664795 sec

Epoch 257 Batch 0 Loss 0.00032
Epoch 257 Loss 0.00448
Time taken for 1 epoch 26.835891008377075

Epoch 330 Batch 0 Loss 0.13574
Epoch 330 Loss 0.46626
Time taken for 1 epoch 27.691344022750854 sec

Epoch 331 Batch 0 Loss 0.10804
Epoch 331 Loss 0.22998
Time taken for 1 epoch 27.015076160430908 sec

Epoch 332 Batch 0 Loss 0.04790
Epoch 332 Loss 0.10090
Time taken for 1 epoch 27.626213312149048 sec

Epoch 333 Batch 0 Loss 0.04223
Epoch 333 Loss 0.05993
Time taken for 1 epoch 27.086085319519043 sec

Epoch 334 Batch 0 Loss 0.02363
Epoch 334 Loss 0.03502
Time taken for 1 epoch 27.636563777923584 sec

Epoch 335 Batch 0 Loss 0.00117
Epoch 335 Loss 0.01813
Time taken for 1 epoch 27.01007652282715 sec

Epoch 336 Batch 0 Loss 0.00291
Epoch 336 Loss 0.01153
Time taken for 1 epoch 27.70356774330139 sec

Epoch 337 Batch 0 Loss 0.00191
Epoch 337 Loss 0.00891
Time taken for 1 epoch 27.000074863433838 sec

Epoch 338 Batch 0 Loss 0.01279
Epoch 338 Loss 0.00817
Time taken for 1 epoch 27.633774757385254 sec

Epoch 339 Batch 0 Loss 0.00204
Epoch 339 Loss 0.00831
Time taken for 1 epoch 27.0340824127197

Epoch 412 Batch 0 Loss 0.00010
Epoch 412 Loss 0.00337
Time taken for 1 epoch 27.369832754135132 sec

Epoch 413 Batch 0 Loss 0.00014
Epoch 413 Loss 0.00313
Time taken for 1 epoch 26.863064527511597 sec

Epoch 414 Batch 0 Loss 0.00372
Epoch 414 Loss 0.00312
Time taken for 1 epoch 27.464188814163208 sec

Epoch 415 Batch 0 Loss 0.00286
Epoch 415 Loss 0.00322
Time taken for 1 epoch 26.85512137413025 sec

Epoch 416 Batch 0 Loss 0.00171
Epoch 416 Loss 0.00363
Time taken for 1 epoch 27.453337907791138 sec

Epoch 417 Batch 0 Loss 0.00008
Epoch 417 Loss 0.00338
Time taken for 1 epoch 26.80702304840088 sec

Epoch 418 Batch 0 Loss 0.00144
Epoch 418 Loss 0.00311
Time taken for 1 epoch 27.45346426963806 sec

Epoch 419 Batch 0 Loss 0.00544
Epoch 419 Loss 0.00321
Time taken for 1 epoch 26.853187799453735 sec

Epoch 420 Batch 0 Loss 0.00419
Epoch 420 Loss 0.00311
Time taken for 1 epoch 27.60598659515381 sec

Epoch 421 Batch 0 Loss 0.00009
Epoch 421 Loss 0.00329
Time taken for 1 epoch 28.11643099784851 

Epoch 494 Batch 0 Loss 0.00830
Epoch 494 Loss 0.00529
Time taken for 1 epoch 27.473096132278442 sec

Epoch 495 Batch 0 Loss 0.00854
Epoch 495 Loss 0.00517
Time taken for 1 epoch 26.81702971458435 sec

Epoch 496 Batch 0 Loss 0.00790
Epoch 496 Loss 0.00415
Time taken for 1 epoch 27.468892812728882 sec

Epoch 497 Batch 0 Loss 0.00123
Epoch 497 Loss 0.00451
Time taken for 1 epoch 26.807093381881714 sec

Epoch 498 Batch 0 Loss 0.00925
Epoch 498 Loss 0.00488
Time taken for 1 epoch 27.47085189819336 sec

Epoch 499 Batch 0 Loss 0.00010
Epoch 499 Loss 0.00512
Time taken for 1 epoch 26.83105206489563 sec

Epoch 500 Batch 0 Loss 0.00012
Epoch 500 Loss 0.00422
Time taken for 1 epoch 27.44649839401245 sec



In [16]:
def evaluate_sentence(sentence):
  sentence = dataset_creator.preprocess_sentence(sentence)

  inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                          maxlen=max_length_input,
                                                          padding='post')
  inputs = tf.convert_to_tensor(inputs)
  inference_batch_size = inputs.shape[0]
  result = ''

  enc_start_state = [tf.zeros((inference_batch_size, units)), tf.zeros((inference_batch_size,units))]
  enc_out, enc_h, enc_c = encoder(inputs, enc_start_state)

  dec_h = enc_h
  dec_c = enc_c

  start_tokens = tf.fill([inference_batch_size], targ_lang.word_index['<start>'])
  end_token = targ_lang.word_index['<end>']

  greedy_sampler = tfa.seq2seq.GreedyEmbeddingSampler()

  # Instantiate BasicDecoder object
  decoder_instance = tfa.seq2seq.BasicDecoder(cell=decoder.rnn_cell, sampler=greedy_sampler, output_layer=decoder.fc)
  # Setup Memory in decoder stack
  decoder.attention_mechanism.setup_memory(enc_out)

  # set decoder_initial_state
  decoder_initial_state = decoder.build_initial_state(inference_batch_size, [enc_h, enc_c], tf.float32)


  ### Since the BasicDecoder wraps around Decoder's rnn cell only, you have to ensure that the inputs to BasicDecoder 
  ### decoding step is output of embedding layer. tfa.seq2seq.GreedyEmbeddingSampler() takes care of this. 
  ### You only need to get the weights of embedding layer, which can be done by decoder.embedding.variables[0] and pass this callabble to BasicDecoder's call() function

  decoder_embedding_matrix = decoder.embedding.variables[0]

  outputs, _, _ = decoder_instance(decoder_embedding_matrix, start_tokens = start_tokens, end_token= end_token, initial_state=decoder_initial_state)
  return outputs.sample_id.numpy()

def translate(sentence):
  result = evaluate_sentence(sentence)
  print(result)
  result = targ_lang.sequences_to_texts(result)
  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))

In [17]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [18]:
translate(u'Peu awak nyan meurasa salah?')

[[  9   8  56 107   6   3]]
Input: Peu awak nyan meurasa salah?
Predicted translation: ['apa kamu merasa bersalah ? <end>']


In [19]:
translate(u'Ahmad cuma meuseunda.')

[[  7 106  34   4   3]]
Input: Ahmad cuma meuseunda.
Predicted translation: ['ahmad cuma bercanda . <end>']


In [20]:
translate(u'Peu Ahmad meuseunda?')

[[ 9  7 34  6  3]]
Input: Peu Ahmad meuseunda?
Predicted translation: ['apa ahmad bercanda ? <end>']


In [21]:
translate(u'Ahmad adalah sidroe siswa.')

[[ 7 20 36 84  4  3]]
Input: Ahmad adalah sidroe siswa.
Predicted translation: ['ahmad adalah seorang siswa . <end>']


In [22]:
translate(u'Lôn beuruntông.')

[[ 5 67 89  4  3]]
Input: Lôn beuruntông.
Predicted translation: ['saya dengan laki . <end>']


In [23]:
translate(u'Nisa adalah sidroe dokter.')

[[69 20 36 69  4  3]]
Input: Nisa adalah sidroe dokter.
Predicted translation: ['nisa adalah seorang nisa . <end>']


In [24]:
translate(u'Bèk kapeu meuen Ahmad.')

[[ 31 290   7  14   3]]
Input: Bèk kapeu meuen Ahmad.
Predicted translation: ['jangan mempermainkan ahmad ! <end>']


In [25]:
translate(u'Lôn adalah sidroe yang gasa.')

[[  5  20  36  15 452   4   3]]
Input: Lôn adalah sidroe yang gasa.
Predicted translation: ['saya adalah seorang yang kasar . <end>']


In [26]:
translate(u'Jih adalah sidroe yang gasa.')

[[ 11  20  36  15 452   4   3]]
Input: Jih adalah sidroe yang gasa.
Predicted translation: ['dia adalah seorang yang kasar . <end>']


In [27]:
translate(u'Jih galak lôn.')

[[11 32  5  4  3]]
Input: Jih galak lôn.
Predicted translation: ['dia suka saya . <end>']


In [28]:
translate(u'Jih nyan soe?')

[[11 13 44  6  3]]
Input: Jih nyan soe?
Predicted translation: ['dia itu siapa ? <end>']


In [29]:
translate(u'Ahmad yang keuphôn.')

[[  7  15 427   4   3]]
Input: Ahmad yang keuphôn.
Predicted translation: ['ahmad yang pertama . <end>']


In [30]:
translate(u'Ahmad hana peu-peu.')

[[ 7 10  9  9  6  3]]
Input: Ahmad hana peu-peu.
Predicted translation: ['ahmad tidak apa apa ? <end>']


In [31]:
translate(u'Ahmad seudéh.')

[[  7 270   4   3]]
Input: Ahmad seudéh.
Predicted translation: ['ahmad sedih . <end>']


In [32]:
translate(u'Lôn kalön Ahmad di miyup nyoe.')

[[  5 102   7  16  49   4   3]]
Input: Lôn kalön Ahmad di miyup nyoe.
Predicted translation: ['saya melihat ahmad di dalam . <end>']


In [33]:
translate(u'Awakkah hana pu-pu?')

[[129  10   9   9   6   3]]
Input: Awakkah hana pu-pu?
Predicted translation: ['kalian tidak apa apa ? <end>']


In [34]:
translate(u'Nyan kön urusan Ahmad.')

[[ 13  33 101   4   3]]
Input: Nyan kön urusan Ahmad.
Predicted translation: ['itu bukan mencobanya . <end>']


In [35]:
translate(u'Nyan urusan lôn.')

[[13 13  5  4  3]]
Input: Nyan urusan lôn.
Predicted translation: ['itu itu saya . <end>']


In [36]:
translate(u'Nyan kön moto lôn.')

[[13 33 74  5  4  3]]
Input: Nyan kön moto lôn.
Predicted translation: ['itu bukan masalah saya . <end>']


In [37]:
translate(u'Lôn hana galak sie lumo.')

[[  5  10  32 776 777   4   3]]
Input: Lôn hana galak sie lumo.
Predicted translation: ['saya tidak suka daging sapi . <end>']


In [38]:
translate(u'Lôn galak sie lumo.')

[[  5  32 217 776   4   3]]
Input: Lôn galak sie lumo.
Predicted translation: ['saya suka makanan daging . <end>']


In [39]:
translate(u'Ahmad galak sie lumo.')

[[  7  32 217   4   3]]
Input: Ahmad galak sie lumo.
Predicted translation: ['ahmad suka makanan . <end>']


In [40]:
translate(u'Lôn ka di peutubiet.')

[[  5  38 778   4   3]]
Input: Lôn ka di peutubiet.
Predicted translation: ['saya sudah dikeluarkan . <end>']


In [41]:
translate(u'Jih ka sep meubantu.')

[[ 11  38  46 121   4   3]]
Input: Jih ka sep meubantu.
Predicted translation: ['dia sudah cukup membantu . <end>']


In [42]:
translate(u'Ahmad hana meubantu.')

[[  7  10 217   4   3]]
Input: Ahmad hana meubantu.
Predicted translation: ['ahmad tidak makanan . <end>']


In [43]:
translate(u'Peukeuh rumôh Ahmad jioh?')

[[ 18 766   7   6   3]]
Input: Peukeuh rumôh Ahmad jioh?
Predicted translation: ['apakah rumahmu ahmad ? <end>']


In [44]:
translate(u'Peukeuh hana beda?')

[[18  7  4  3]]
Input: Peukeuh hana beda?
Predicted translation: ['apakah ahmad . <end>']


In [45]:
translate(u'Ahmad hana galak bôh manok.')

[[  7  10  32 237   4   3]]
Input: Ahmad hana galak bôh manok.
Predicted translation: ['ahmad tidak suka telur . <end>']


In [46]:
translate(u'Ahmad galak bôh manok.')

[[  7  32 534   4   3]]
Input: Ahmad galak bôh manok.
Predicted translation: ['ahmad suka terung . <end>']


In [47]:
translate(u'Peukeuh Ahmad na sitangèn?')

[[ 18   7  25 200   6   3]]
Input: Peukeuh Ahmad na sitangèn?
Predicted translation: ['apakah ahmad punya sepeda ? <end>']


In [48]:
translate(u'Ahmad bèk peugah aba!')

[[  7  20 128   5  14   3]]
Input: Ahmad bèk peugah aba!
Predicted translation: ['ahmad adalah kakak saya ! <end>']


In [49]:
translate(u'Nisa mantöng golom meuphom?')

[[69 39 65 70  6  3]]
Input: Nisa mantöng golom meuphom?
Predicted translation: ['nisa masih belum mengerti ? <end>']


In [50]:
translate(u'Ahmad perlè pèng.')

[[  7 186  66   4   3]]
Input: Ahmad perlè pèng.
Predicted translation: ['ahmad membutuhkan uang . <end>']


In [51]:
translate(u'Teurimeng geunaseh Ahmad.')

[[53 51  7  4  3]]
Input: Teurimeng geunaseh Ahmad.
Predicted translation: ['terima kasih ahmad . <end>']


In [52]:
translate(u'Jih adalah sidroe yang gasa.')

[[ 11  20  36  15 452   4   3]]
Input: Jih adalah sidroe yang gasa.
Predicted translation: ['dia adalah seorang yang kasar . <end>']


In [53]:
translate(u'Awak nyan akan meutarông.')

[[ 30  19 227   4   3]]
Input: Awak nyan akan meutarông.
Predicted translation: ['mereka akan bertarung . <end>']


In [54]:
translate(u'Ahmad poh lôn.')

[[  7 425   5   4   3]]
Input: Ahmad poh lôn.
Predicted translation: ['ahmad memukul saya . <end>']


In [55]:
translate(u'Kalön makanan di miyup nyoe.')

[[138 794 144  12   4   3]]
Input: Kalön makanan di miyup nyoe.
Predicted translation: ['lihat ulang tahun ini . <end>']


In [56]:
translate(u'Jih mantöng golom meuphom.')

[[11 39 65 70  4  3]]
Input: Jih mantöng golom meuphom.
Predicted translation: ['dia masih belum mengerti . <end>']


In [57]:
translate(u'Peu Ahmad meuen boling?')

[[  9   7 136 440   6   3]]
Input: Peu Ahmad meuen boling?
Predicted translation: ['apa ahmad bermain boling ? <end>']


In [58]:
translate(u'Peu awak nyan meuen boling?')

[[  9   8 136 440   6   3]]
Input: Peu awak nyan meuen boling?
Predicted translation: ['apa kamu bermain boling ? <end>']


In [59]:
translate(u'Peu awakkah meuen boling?')

[[  9 129 318   6   3]]
Input: Peu awakkah meuen boling?
Predicted translation: ['apa kalian mobilmu ? <end>']


In [60]:
translate(u'Bèk kapeumeuen jih!')

[[ 31 690  14   3]]
Input: Bèk kapeumeuen jih!
Predicted translation: ['jangan sentuh ! <end>']


In [61]:
translate(u'Jih that menyesal.')

[[ 11  24 627   4   3]]
Input: Jih that menyesal.
Predicted translation: ['dia sangat berantakan . <end>']


In [62]:
translate(u'Jih hana menyesal.')

[[11 10 89  4  3]]
Input: Jih hana menyesal.
Predicted translation: ['dia tidak laki . <end>']


In [63]:
translate(u'Awak nyan kalön Ahmad.')

[[ 30 258   7   4   3]]
Input: Awak nyan kalön Ahmad.
Predicted translation: ['mereka dulunya ahmad . <end>']


In [64]:
translate(u'Ahmad kalön jih.')

[[  7 102  11   4   3]]
Input: Ahmad kalön jih.
Predicted translation: ['ahmad melihat dia . <end>']


In [65]:
translate(u'Ahmad hana bahgia')

[[  7  10 103   4   3]]
Input: Ahmad hana bahgia
Predicted translation: ['ahmad tidak bahagia . <end>']


In [66]:
translate(u'Lôn hana bahgia.')

[[  5  10 103   4   3]]
Input: Lôn hana bahgia.
Predicted translation: ['saya tidak bahagia . <end>']


In [67]:
translate(u'Jih bahgia ngön Ahmad.')

[[ 11 564  67   7   4   3]]
Input: Jih bahgia ngön Ahmad.
Predicted translation: ['dia hanya dengan ahmad . <end>']


In [68]:
translate(u'Kah bakdeuh tingeh saket.')

[[  8 309  28  60   4   3]]
Input: Kah bakdeuh tingeh saket.
Predicted translation: ['kamu tampak sedang sakit . <end>']


In [69]:
translate(u'Ahmad hana teumpat tinggai.')

[[  7  10 180  96   4   3]]
Input: Ahmad hana teumpat tinggai.
Predicted translation: ['ahmad tidak tempat tinggal . <end>']


In [70]:
translate(u'Hana yang kalön awak nyan.')

[[ 10  21  15 102  30   4   3]]
Input: Hana yang kalön awak nyan.
Predicted translation: ['tidak ada yang melihat mereka . <end>']


In [71]:
translate(u'Peudeuh nyan bak Ahmad.')

[[308  13  71   7   4   3]]
Input: Peudeuh nyan bak Ahmad.
Predicted translation: ['pasti itu pada ahmad . <end>']


In [72]:
translate(u'Peudeuh nyan bak jih.')

[[208  13  71  11   4   3]]
Input: Peudeuh nyan bak jih.
Predicted translation: ['perlihatkan itu pada dia . <end>']


In [73]:
translate(u'Bèk peudeuh nyan bak lôn.')

[[ 31 208   5   4   3]]
Input: Bèk peudeuh nyan bak lôn.
Predicted translation: ['jangan perlihatkan saya . <end>']


In [74]:
translate(u'Bèk peudeuh nyan bak jih.')

[[ 31 208  13   4   3]]
Input: Bèk peudeuh nyan bak jih.
Predicted translation: ['jangan perlihatkan itu . <end>']


In [75]:
translate(u'Bèk peuleuh Ahmad.')

[[ 31 280   7   4   3]]
Input: Bèk peuleuh Ahmad.
Predicted translation: ['jangan lupakan ahmad . <end>']


In [76]:
translate(u'Jih i peusalah Ahmad.')

[[ 11 570   7   4   3]]
Input: Jih i peusalah Ahmad.
Predicted translation: ['dia supir ahmad . <end>']


In [77]:
translate(u'Jih meurasa hana bahgia.')

[[ 11  56  10 103   4   3]]
Input: Jih meurasa hana bahgia.
Predicted translation: ['dia merasa tidak bahagia . <end>']


In [78]:
translate(u'Jih hana beuruntông.')

[[ 11  10 285   4   3]]
Input: Jih hana beuruntông.
Predicted translation: ['dia tidak haus . <end>']


In [79]:
translate(u'Awak nyan beuruntông.')

[[ 30 194   4   3]]
Input: Awak nyan beuruntông.
Predicted translation: ['mereka merah . <end>']


In [80]:
translate(u'Lôn yee keu mate.')

[[  5 189  71 738   4   3]]
Input: Lôn yee keu mate.
Predicted translation: ['saya takut pada kematian . <end>']


In [81]:
translate(u'Ahmad akui jih salah.')

[[  7 629  11 455   4   3]]
Input: Ahmad akui jih salah.
Predicted translation: ['ahmad akui dia mengenalnya . <end>']


In [82]:
translate(u'Ahmad hana patéh keu Nisa.')

[[  7  10 117   4   3]]
Input: Ahmad hana patéh keu Nisa.
Predicted translation: ['ahmad tidak percaya . <end>']


In [83]:
translate(u'Lôn hana patéh keu Ahmad.')

[[  5  10 117  78   4   3]]
Input: Lôn hana patéh keu Ahmad.
Predicted translation: ['saya tidak percaya padamu . <end>']


In [84]:
translate(u'Lôn patéh keu Ahmad.')

[[  5 117  20   4   3]]
Input: Lôn patéh keu Ahmad.
Predicted translation: ['saya percaya adalah . <end>']


In [85]:
translate(u'Lôn hawa meureumpök ngön kah.')

[[  5  76  48 256   4   3]]
Input: Lôn hawa meureumpök ngön kah.
Predicted translation: ['saya ingin bertemu dengannya . <end>']


In [86]:
translate(u'Ahmad hawa meureumpök ngön jih.')

[[  7  61  48 256   4   3]]
Input: Ahmad hawa meureumpök ngön jih.
Predicted translation: ['ahmad mau bertemu dengannya . <end>']


In [87]:
translate(u'Lôn hawa meureumpök jih.')

[[ 5 76 48 11  4  3]]
Input: Lôn hawa meureumpök jih.
Predicted translation: ['saya ingin bertemu dia . <end>']


In [88]:
translate(u'Ahmad na di dalam moto.')

[[ 7 21 16 49 68  4  3]]
Input: Ahmad na di dalam moto.
Predicted translation: ['ahmad ada di dalam mobil . <end>']


In [89]:
translate(u'Ahmad sidroe yang hana sempurna.')

[[  7  36  15 115   4   3]]
Input: Ahmad sidroe yang hana sempurna.
Predicted translation: ['ahmad seorang yang sempurna . <end>']


In [90]:
translate(u'Hana yang tanyöng bak jih.')

[[ 10  21  15 248  78   4   3]]
Input: Hana yang tanyöng bak jih.
Predicted translation: ['tidak ada yang bertanya padamu . <end>']


In [91]:
translate(u'Peu yang peugöt jih jaga?')

[[  9  15 607 608   6   3]]
Input: Peu yang peugöt jih jaga?
Predicted translation: ['apa yang membuatmu terbangun ? <end>']


In [92]:
translate(u'Peu yang peugöt kah hana jaga?')

[[  9  15 607 608   6   3]]
Input: Peu yang peugöt kah hana jaga?
Predicted translation: ['apa yang membuatmu terbangun ? <end>']


In [93]:
translate(u'Pakön tanyoe harus bayeu?')

[[154  23  40  74   6   3]]
Input: Pakön tanyoe harus bayeu?
Predicted translation: ['mengapa kita harus masalah ? <end>']


In [94]:
translate(u'Pakön jih hana bayeu?')

[[155  11  10 711   6   3]]
Input: Pakön jih hana bayeu?
Predicted translation: ['ia dia tidak asing ? <end>']


In [95]:
translate(u'Jih hana deungö.')

[[ 11  10  89 156  11   4   3]]
Input: Jih hana deungö.
Predicted translation: ['dia tidak laki mempercayai dia . <end>']


In [96]:
translate(u'Peu awak nyan mandum ka siap?')

[[  9   8 132 332   6   3]]
Input: Peu awak nyan mandum ka siap?
Predicted translation: ['apa kamu semua siap ? <end>']


In [97]:
translate(u'Jih hana pu-pu?')

[[11 10  9  9  6  3]]
Input: Jih hana pu-pu?
Predicted translation: ['dia tidak apa apa ? <end>']


In [98]:
translate(u'Bèk kapeu keucewa jih.')

[[ 31 199  11   4   3]]
Input: Bèk kapeu keucewa jih.
Predicted translation: ['jangan mengecewakan dia . <end>']


In [99]:
translate(u'Bèk kapeu keucewa kamoe.')

[[ 31 199  29   4   3]]
Input: Bèk kapeu keucewa kamoe.
Predicted translation: ['jangan mengecewakan kami . <end>']


In [100]:
translate(u'Bèk kapeu keucewa awak nyan.')

[[ 31 199  30   4   3]]
Input: Bèk kapeu keucewa awak nyan.
Predicted translation: ['jangan mengecewakan mereka . <end>']


In [101]:
translate(u'Nisa mantöng muda.')

[[ 69  39 124   4   3]]
Input: Nisa mantöng muda.
Predicted translation: ['nisa masih muda . <end>']


In [102]:
translate(u'Awak nyan mantöng golom i noe.')

[[30 39 65 16 37  4  3]]
Input: Awak nyan mantöng golom i noe.
Predicted translation: ['mereka masih belum di sini . <end>']


In [103]:
translate(u'Ahmad kön dokter.')

[[  7  33 234   4   3]]
Input: Ahmad kön dokter.
Predicted translation: ['ahmad bukan dokter . <end>']


In [104]:
translate(u'Lôn galak bôh manok.')

[[  5  32 534   4   3]]
Input: Lôn galak bôh manok.
Predicted translation: ['saya suka terung . <end>']


In [105]:
translate(u'Satnyoe jih kalön Ahmad.')

[[635  11 109   7   4   3]]
Input: Satnyoe jih kalön Ahmad.
Predicted translation: ['barusan dia hari ahmad . <end>']


In [106]:
translate(u'Peukeuh Nisa mantöng saket?')

[[18 69 39 60  6  3]]
Input: Peukeuh Nisa mantöng saket?
Predicted translation: ['apakah nisa masih sakit ? <end>']


In [107]:
translate(u'Hobi jih musik.')

[[645  11 646   4   3]]
Input: Hobi jih musik.
Predicted translation: ['hobi dia musik . <end>']


In [108]:
translate(u'Jih hobi musik.')

[[626 560   4   3]]
Input: Jih hobi musik.
Predicted translation: ['kamarnya ketemu . <end>']


In [109]:
translate(u'Lôn ureueng yang hana sempurna.')

[[  5 728  15 115   4   3]]
Input: Lôn ureueng yang hana sempurna.
Predicted translation: ['saya paman yang sempurna . <end>']


In [110]:
translate(u'Ahmad mantöng bungeh.')

[[ 7 39  4  3]]
Input: Ahmad mantöng bungeh.
Predicted translation: ['ahmad masih . <end>']


In [111]:
translate(u'Lôn hana peusalah Ahmad.')

[[  5  10 632   4   3]]
Input: Lôn hana peusalah Ahmad.
Predicted translation: ['saya tidak menyalahkanmu . <end>']


In [112]:
translate(u'Jih meurasa göt uroe nyoe.')

[[11 56 67 12  4  3]]
Input: Jih meurasa göt uroe nyoe.
Predicted translation: ['dia merasa dengan ini . <end>']


In [113]:
translate(u'Jih perle keurtah.')

[[ 11  76 352   4   3]]
Input: Jih perle keurtah.
Predicted translation: ['dia ingin kertas . <end>']


In [114]:
translate(u'Lôn hawa meureumpök ngön awak nyan.')

[[  5  76  48 256   4   3]]
Input: Lôn hawa meureumpök ngön awak nyan.
Predicted translation: ['saya ingin bertemu dengannya . <end>']


In [115]:
translate(u'Awak nyan that deuek.')

[[ 30 324 627   4   3]]
Input: Awak nyan that deuek.
Predicted translation: ['mereka panas berantakan . <end>']


In [116]:
translate(u'Jih na di dalam moto.')

[[11 21 16 49 68  4  3]]
Input: Jih na di dalam moto.
Predicted translation: ['dia ada di dalam mobil . <end>']


In [117]:
translate(u'Kubah jih sidroe!')

[[187  11 127  14   3]]
Input: Kubah jih sidroe!
Predicted translation: ['tinggalkan dia sendiri ! <end>']


In [118]:
translate(u'Jih lagak.')

[[ 11 148   4   3]]
Input: Jih lagak.
Predicted translation: ['dia cantik . <end>']
